**Logging** is an important module in tracking the work or performance. Maintaining logs helps is tracing the error, variable value and other many uses.

**DXC industrialized AI** starter also performs default logging for few crucial functions. We have provided the automatic logging functionality for below major funtions in this library.



1.   Build Data Pipelines
2.   Run AI Experiments
3.   Publish Microservice


These auto logging is introduced to save the user build pipelines, inputs provided for run experiment function and also to store the microservice design skeleton used for publishing model in algorithmia.

These logs can be referred back in case to rerun or reuse any pipeline or microservice design details. Logs will be saved in the same location where notebook with functions used. 

Log files can be viewed from left side files folder (in google colab) or files will be same folder where script is executed (in juypter).

We have also provided commands below to view logs files.

In [ ]:
##You must restart the runtime/session in order to use newly installed versions.
%%capture
! pip install DXC-Industrialized-AI-Starter
import os
os.kill(os.getpid(), 9)

In [ ]:
%%capture
from dxc import ai

In [ ]:
# TODO: Access raw data.
##reads json from a url and flattens it into a dataframe
df = ai.read_data_frame_from_remote_json("https://data.cincinnati-oh.gov/resource/ucjy-ykv4.json")
df.head()

,request_id,request_status,funding_source,request_type,department_name,fiscal_year,replacement_body_style,equipment_class,equip_id_to_replace,replacement_make,replacement_model,date_fleet_doc_entered,fleet_document_type,purchasing_bid_date,est_unit_cost,date_bid_closed,date_po_awarded,actual_unit_cost,procurement_plan
0,463,Completed,CAP,Fleet Procurement Plan,Fire,2019,SEDAN,PC-GS,06193,CHEVROLET,MALIBU,2019-04-09T00:00:00.000,RQS,2019-07-05T00:00:00.000,19246,2019-07-05T00:00:00.000,2019-07-10T00:00:00.000,17507.14,Passenger Cars
1,674,Completed,Auction Funds,Supplemental Request,DPS - Fleet Services Division,2019,HYDRANT PUMP,NaN,NaN,MILWAUKEE,M18 LI-ION,2019-11-05T00:00:00.000,DO,NaN,400,NaN,NaN,355.55,Specialized Small
2,688,Denied,Operating Funds,Supplemental Request,DPS - Neighborhood Operations Division,2020,NaN,GROUNDS,NEW,NaN,NaN,NaN,DO,NaN,4631.8,NaN,NaN,NaN,NaN
3,818,Completed,Other,Supplemental Request,Police,2017,SUV,PC-GS,TBD,Chevrolet,Traverse,2020-06-01T00:00:00.000,DO,NaN,25799.5,NaN,NaN,25799.5,Passenger Cars
4,248,Completed,CIP,Fleet Procurement Plan,Recreation,2017,HEDGE TRIMMER,SPECIALIZED SMALL,46500,STIHL,"HS82T(24"")",2017-08-01T00:00:00.000,DO,NaN,250,NaN,2017-08-07T00:00:00.000,384.96,Horticulture


In [ ]:
# TODO: define the data fields
text_fields = []
date_fields = ['date_fleet_doc_entered', 'purchasing_bid_date', 'date_bid_closed', 'date_po_awarded']
numeric_fields = ['est_unit_cost', 'actual_unit_cost']
categorical_fields = ['request_id','request_status', 'funding_source', 'request_type', 'department_name', 
                      'fiscal_year', 'replacement_body_style','equipment_class','equip_id_to_replace',
                      'replacement_make', 'replacement_model','fleet_document_type', 'procurement_plan']


In [ ]:
#clean the data
impute = True
raw_data = ai.clean_dataframe(df, impute, text_fields, date_fields, numeric_fields, categorical_fields)

#display excerpts of the raw data
raw_data.head()

Imputing row 1/862 with 0 missing, elapsed time: 0.163
Imputing row 101/862 with 0 missing, elapsed time: 0.169
Imputing row 201/862 with 0 missing, elapsed time: 0.173
Imputing row 301/862 with 2 missing, elapsed time: 0.176
Imputing row 401/862 with 1 missing, elapsed time: 0.181
Imputing row 501/862 with 0 missing, elapsed time: 0.184
Imputing row 601/862 with 0 missing, elapsed time: 0.187
Imputing row 701/862 with 0 missing, elapsed time: 0.191
Imputing row 801/862 with 0 missing, elapsed time: 0.195


,request_id,request_status,funding_source,request_type,department_name,fiscal_year,replacement_body_style,equipment_class,equip_id_to_replace,replacement_make,replacement_model,date_fleet_doc_entered,fleet_document_type,purchasing_bid_date,est_unit_cost,date_bid_closed,date_po_awarded,actual_unit_cost,procurement_plan
0,463,Completed,CAP,Fleet Procurement Plan,Fire,2019,SEDAN,PC-GS,06193,CHEVROLET,MALIBU,2019-04-09T00:00:00+00:00,RQS,2019-07-05T00:00:00+00:00,19246.0,2019-07-05T00:00:00+00:00,2019-07-10T00:00:00+00:00,17507.14,Passenger Cars
1,674,Completed,Auction Funds,Supplemental Request,DPS - Fleet Services Division,2019,HYDRANT PUMP,OTHER,NA,MILWAUKEE,M18 LI-ION,2019-11-05T00:00:00+00:00,DO,2019-07-31T00:00:00+00:00,400.0,2019-07-05T00:00:00+00:00,2019-10-31T00:00:00+00:00,355.55,Specialized Small
2,688,Denied,Operating Funds,Supplemental Request,DPS - Neighborhood Operations Division,2020,Backpack blower,GROUNDS,NEW,WOODS,MALIBU,2019-12-03T00:00:00+00:00,DO,2019-12-31T00:00:00+00:00,4631.8,2019-02-27T00:00:00+00:00,2020-01-06T00:00:00+00:00,4785.25,Passenger Cars
3,818,Completed,Other,Supplemental Request,Police,2017,SUV,PC-GS,TBD,Chevrolet,Traverse,2020-06-01T00:00:00+00:00,DO,2020-04-29T00:00:00+00:00,25799.5,2020-06-19T00:00:00+00:00,2020-06-12T00:00:00+00:00,25799.50,Passenger Cars
4,248,Completed,CIP,Fleet Procurement Plan,Recreation,2017,HEDGE TRIMMER,SPECIALIZED SMALL,46500,STIHL,"HS82T(24"")",2017-08-01T00:00:00+00:00,DO,2018-03-02T00:00:00+00:00,250.0,2018-04-11T00:00:00+00:00,2017-08-07T00:00:00+00:00,384.96,Horticulture


In [ ]:
# TODO: specify the details of the data layer
data_layer = {
    "connection_string": "<your connection_string>",
    "collection_name": "<your collection_name>",
    "database_name": "<your database_name>",
    "data_source":"<Source of your datset>",
    "cleaner":"<whether applied cleaner yes/no >"
}

In [ ]:
wrt_raw_data = ai.write_raw_data(data_layer, raw_data, date_fields)

In [ ]:
# TODO: define the code needed to refine the raw data
def data_pipeline():

  pipe = [
          {
              '$group':{
                  '_id': {
                      "funding_source":"$Funding_Source",
                      "request_type":"$Request Type",
                      "department_name":"$Department Name",
                      "replacement_body_style":"$Replacement_Body Style",
                      "equipment_class":"$equipment Class",
                      "replacement_make":"$replacement Make",
                      "replacement_model":"$REPlacement model",
                      "procurement_plan":"$Procurement Plan"
                      },
                  "avg_est_unit_cost":{"$avg":"$Est Unit Cost"},
                  "avg_est_unit_cost_error":{"$avg":{ "$subtract": [ "$Est Unit Cost", "$Actual Unit Cost" ] }}
              }
          }
  ]

  return pipe


df = ai.access_data_from_pipeline(wrt_raw_data, data_pipeline())
df.head()

,avg_est_unit_cost,avg_est_unit_cost_error,_id.funding_source,_id.request_type,_id.department_name,_id.replacement_body_style,_id.equipment_class,_id.replacement_make,_id.replacement_model,_id.procurement_plan
0,18500.000000,-550.613333,CAP,Fleet Procurement Plan,Police,SEDAN,PC-GS,FORD,FUSION,Passenger Cars
1,122100.000000,99303.922000,CIP,Fleet Procurement Plan,Recreation,HANDHELD BLOWER,SPECIALIZED SMALL,STIHL,BG86,Horticulture
2,29508.000000,1619.300000,CIP,Fleet Procurement Plan,Fire,SEDAN,PC-GS,NISSAN,LEAF,Passenger Cars
3,21000.000000,3492.860000,CAP,Fleet Procurement Plan,Police,SEDAN,PC-GS,CHEVROLET,MALIBU,Passenger Cars
4,177505.176667,-33373.490000,CIP,Fleet Procurement Plan,Fire,AMBULANCE,AMBULANCE,FORD,F-450,Medic Units


In [ ]:
!cat pipeline.log

2020-09-25 17:25:46,500:dxc.ai.logging.pipeline_logging:INFO:[{'$group': {'_id': {'funding_source': '$Funding_Source', 'request_type': '$Request Type', 'department_name': '$Department Name', 'replacement_body_style': '$Replacement_Body Style', 'equipment_class': '$equipment Class', 'replacement_make': '$replacement Make', 'replacement_model': '$REPlacement model', 'procurement_plan': '$Procurement Plan'}, 'avg_est_unit_cost': {'$avg': '$Est Unit Cost'}, 'avg_est_unit_cost_error': {'$avg': {'$subtract': ['$Est Unit Cost', '$Actual Unit Cost']}}}}]


In [ ]:
# TODO: design and run an experiment
experiment_design = {
    #model options include ['regression()', 'classification()']
    "model": ai.regression(),
    "labels": df.avg_est_unit_cost_error,
    "data": df,
    #Tell the model which column is 'output'
    #Also note columns that aren't purely numerical
    #Examples include ['nlp', 'date', 'categorical', 'ignore']
    "meta_data": {
      "avg_est_unit_cost_error": "output",
      "_id.funding_source": "categorical",
      "_id.department_name": "categorical",
      "_id.replacement_body_style": "categorical",
      "_id.replacement_make": "categorical",
      "_id.replacement_model": "categorical",
      "_id.procurement_plan": "categorical"
  }
}
trained_model = ai.run_experiment(experiment_design)

None


***********************************************
Advanced scoring metrics for the trained regression model on this particular dataset:

Here is the overall RMSE for these predictions:
52000.602795306

Here is the average of the predictions:
2270.1296244803493

Here is the average actual value on this validation set:
5177.398634004884

Here is the median prediction:
34.449001049076

Here is the median actual value:
0.0

Here is the mean absolute error:
22357.1842945423

Here is the median absolute error (robust to outliers):
4072.8118905359233

Here is the explained variance:
0.09295957963027268

Here is the R-squared value:
0.09011551184665778
Count of positive differences (prediction > actual):
44
Count of negative differences:
22
Average positive difference:
14587.43646376332
Average negative difference:
-37896.67995610025


***********************************************




In [ ]:
!cat experiment_design.log

2020-09-25 17:25:53,222:dxc.ai.logging.experiment_design_logging:INFO:{'model': <dxc.ai.run_model.run_model.regression object at 0x7fd4e39d9860>, 'labels': 0        -550.613333
1       99303.922000
2        1619.300000
3        3492.860000
4      -33373.490000
           ...      
323    186760.050000
324      1118.745000
325      4543.240000
326     -4894.000000
327      3627.664000
Name: avg_est_unit_cost_error, Length: 328, dtype: float64, 'data':      avg_est_unit_cost  ...        _id.procurement_plan
0         18500.000000  ...              Passenger Cars
1        122100.000000  ...                Horticulture
2         29508.000000  ...              Passenger Cars
3         21000.000000  ...              Passenger Cars
4        177505.176667  ...                 Medic Units
..                 ...  ...                         ...
323      187000.000000  ...                       Other
324       28500.000000  ...           Specialized Small
325       30000.000000  ...           Spe

In [ ]:
#TODO design a microservice
microservice_design = {
    "microservice_name": "<Name of your microservice>",
    "microservice_description": "<Brief description about your microservice>",
    "execution_environment_username": "<Algorithmia username>",
    "api_key": "<your api_key>",
    "api_namespace": "<your api namespace>",   
    "model_path":"<your model_path>"
}

In [ ]:
# publish the micro service and display the url of the api
api_url = ai.publish_microservice(microservice_design, trained_model, verbose = False)
print("api url: " + api_url)

api url: https://api.algorithmia.com/v1/algo/mbandru2/dxcaistarter/0.1.0


In [ ]:
!cat microservice_design_log.log

2020-09-25 17:26:10,572:dxc.ai.logging.microservice_logging:INFO:{'microservice_name': 'dxcaistarter', 'microservice_description': 'test api generated from the DXC ai starter', 'execution_environment_username': 'mbandru2', 'api_key': 'sim1CBEvUbMpducHofhsUQIJBx41', 'api_namespace': 'mbandru2/dxcaistarter', 'model_path': 'data://.my/mycollection'}
